## Raspagem dos resultados das provas da Maratona do Rio 2019 
Meia Maratona, Maratona, Desafio Cidade Maravilhosa, Prova de 5 e 10km e Maratoninha Gloob

In [1]:
import os
import mechanize
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from time import time
from time import sleep
from random import randint

In [2]:
url_base = 'https://globalcronometragem.com.br/evento/'
DATASET_FOLDER  = 'dataset'
br = mechanize.Browser()
br.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.63 Safari/535.7')]

### Meia Maratona

In [3]:
MEIA_MARATONA = "meiadorio2019"

In [7]:
categorias_meia = get_categorias(MEIA_MARATONA)

In [8]:
url_individuais = get_url_resultados_individuais(categorias_meia, MEIA_MARATONA)

--> Request Masculino de 30 ate 34 anos 8.00285792350769
15011 - STANLEY KIPCHIRCHIR KOECH
15008 - GILMAR SILVESTRE LOPES
15001 - ANDRÃ LUIZ SILVA ANTONIO
15015 - JACOB KEMBOI KIPROTICH
24207 - GIOVANI GERONASSO
25848 - JOSIAS NEPOMUCENO RIBEIRO
12370 - PAULO HENRIQUE DA SILVA
19690 - ESTEFANO STUBINSKI FILHO
13594 - GABRIEL PINHEIRO PAIM
13272 - DEIVIDE DE SA E SILVA SANTOS
17672 - DANIEL ZAMBOTTO
15249 - GUILHERME AFFONSO FERREIRA FILHO
24781 - FELIPE BRANCO
13528 - MOISES LIVRAMENTOFELICIDADE
26336 - SERGIO BRUNO ALVES FERREIRA
26606 - ROBERTO FERRARA BUSCATTI
16550 - RENAN BARNABE
13099 - FELIPE DOS SANTOS PONTES
20685 - ANDRE GEWEHR
15909 - ROBERTO SOBREIRA
21733 - MARCELO LOPES SARMENTO
28660 - EVERTON RODOLFO RODRIGUES
13269 - KAIO ROCHA FERREIRA
23604 - BRUNO AUGUSTO SALGADO
28337 - OMAR TRONCOSO ESPINOSA
24632 - CONRADO MARTINEZ MORNAGHI
27500 - RENATO SIMONETTI
25393 - RODRIGO DA SILVA ANDREOLLI
12987 - MOIZES CLODOALDO PAPA FILHO
13606 - JULIO CESAR SA DIOGO
11782 - ADRIANO

22335 - GUILHERME DA SILVA DOS SANTOS
21537 - RODRIGO SCHARDOSIN DE BRITO
20668 - WILSON BASTOS VIEIRA NETO
22172 - BRUNO MAXIMO MALDONADO
24866 - MICHEL YOUSSIF CHAOWICHE
23617 - MARLON BRUNO LARA DE SOUZA
17052 - LUCIANO VAZ DE LIMA
23322 - FRANCISCO DO NASCIMENTO PRADO NETO
25138 - FERNANDO AUGUSTO DE BASTOS GOMES JUNIOR
16870 - LEANDRO SILVA DE OLIVEIRA
25134 - THIAGO MOREIRA ROMUALDO SILVA
12490 - RAFAEL BELEM
11725 - RENATO PERROTTA DE SOUZA
13330 - EVERTON WILSON DOS SANTOS
21779 - DIOGENES DE SOUSA FERREIRA FILHO
25787 - DANIEL RODRIGO LINS DOMICIO
23472 - JAIRO LOPES
23441 - CESAR HENRIQUE LONGUINI
22927 - RAFAEL SUTTER CASTELO
17019 - JOAO RODRIGUES
20007 - CARLOS VINICIUS BANDEIRA DO N DE LIRA
23222 - RAFAEL AOTA TAKENOBU
17561 - ANDRE FELIPE CRISPIM DE OLIVEIRA SANTOS
27218 - LUIZ MARQUES CAMPELO
17063 - DAVID FIRMINO DUTRA
16185 - VINICIUS GOMES DE OLIVEIRA PEREIRA
21000 - LUCAS BENJAMIM AMARAL OLIVEIRA
11799 - GABRIEL PINTO RIBEIRO
12275 - VICTOR HUGO
16710 - RONALDO SAVA

--> Request Masculino de 35 ate 39 anos 7.004485607147217
15004 - VAGNER DA SILVA NORONHA
15031 - VANILSON NEVES
22619 - FAGNER DE PADUA PEREIRA
18181 - LUCAS SIMOES DE ALBUQUERQUE
28484 - VAUNALDO FERREIRA DOS SANTOS
15009 - LEANDRO MELO DE SOUZA
21179 - DUDU BARBOSA
13955 - ANDREI ACHCAR ALBUQUERQUE MARANHAO
13765 - GUSTAVO DA SILVA FERNANDES
16100 - RUY CARMONA SANTOS JUNIOR
26004 - BRUNO LOPES DE ARAUJO DIAS
13662 - BRUNO XAVIER DE LIMA
28485 - HILÃRIO BARROS TELES
23605 - JEFFERSON BARBOSA DE SOUZA
17528 - CAIO VIANA
22506 - ALESSANDRO INACIO
12280 - CARLOS EDUARDO SIMOES MARTINS
19728 - RICARDO STUBER
12655 - WAGNER TEIXEIRA MARTINS
28077 - GUALDONI LOPEZ FRANCISCO JOSE
25312 - LEANDRO BRAZ
26383 - THIAGO LIMA MASSARA
12649 - HENRIQUE MARCELO MAYER
18598 - MARCUS VINICIUS DE SOUZA SALVINO
16284 - EDUARDO TUNISSE
25380 - FELIPE QUADROTTI OTSURU
12335 - ROBERTO DOS SANTOS SILVA
12876 - LEONARDO MACIEL
25398 - EDUARDO GONCALVES RODRIGUES
12129 - ALEXANDRE TAVARES DA SILVA
12886 - J

28357 - JORGE ELIAS MAIDAH
27417 - MARCELO MEIRELES GOULART
12064 - RODRIGO SALES DE MORAIS
22783 - RICARDO DIAS DE PAULA CASTRO
27761 - DIEGO ROCHA BARROZO
27383 - GLAUCO MAXIMIANO DE CAMPOS
24204 - AMIR FERNANDES BORGES
23919 - JOSE MARIA DA SILVA SOUSA JUNIOR
12614 - ALMIR GOMES
27481 - ROBERTO PAULINO MAGALHAES
25112 - IGOR FERNANDES PEREIRA
25962 - ADEMILTON FERNANDO DOURADO SANTANA
20292 - TIAGO LAGOS
22521 - RICARDO RAMOS
27724 - RICARDO ERLICH
12146 - ELIEL ALVES DE ALMEIDA
18972 - PAULO ROBERTO CHAGAS OLIVEIRA
23323 - MARCELO FERREIRA DA SILVA
28595 - GUILHERME PILETTI
18976 - RAPHAEL GONCALVES DOS SANTOS FARIAS
19339 - FABIO GOMES PRIETO
12962 - SELMO DA SILVA ROCHA JUNIOR
19291 - WEIKSON JOSE SILVA
24913 - THIAGO PEIXOTO NUNES
16546 - CEZANE TACIO FERRARI MOSTARO
12911 - MAYKEL FERREIRA RIBEIRO
20978 - DJALMA ANSELMO JUNIOR
26979 - RAFAEL REINA
19938 - ANDRE BERG
20414 - AMAURY SERGIO DE GODOY MAFRA
4848 - DALTON ISAC TONHOLO
20194 - GUSTAVO MACHADO DRUMOND
12548 - ELIAS SAN

--> Request Masculino de 20 ate 24 anos 6.00573468208313
15017 - SENDEKU ALELGN AMOGNE
13288 - FILIPE OLIVEIRA DE JESUS
16335 - BRUNO NOGUEIRA
15227 - TARCISIO FARIA BORGES
21433 - FREDSON ARAGAO PEREIRA JUNIOR
13376 - MARCIO MACHADO
22643 - HIGOR HENRIQUE JORDANO DA SILVA
18050 - ITALO JUNIOR DE OLIVEIRA FELETTI
22915 - DANIEL RICARDO VELLOSO
23473 - ENRICO CAPODEFERRO TORRICELLI
21466 - DANIEL TONELI MENDES
24492 - HADSON CALDAS RODRIGUES
26757 - LUIS FELIPE MOULATLET MARCONDES
28606 - JOAO LUCAS DE SOUZA VICENTE
23941 - ROMULO GUIMARAES
22687 - THASSIO FILIPE ROCHA MAGALHAES
15928 - JEAN WOOLMAY DENSON PIERRE
22648 - AGNALDO FEITOSA DA SILVA JUNIOR
27654 - ARTUR RUEDA MUNERATO
17969 - JOAO GABRIEL DOS SANTOS XAVIER
13578 - ANDREW GABRIEL ARAUJO CORREA
28889 - ANDRE LUIS DA COSTA
25038 - LENNON OREQUES
21534 - MATEUS FREITAS CARDEAL DE ARAUJO
24327 - CLAUDIO ALVES DA CRUZ JUNIOR
23713 - CARLOS FRANCISCO DE ARAUJO FREIRE
23721 - ARTHUR RODRIGUES DA COSTA FILIZZOLA
28545 - MAURICIO GON

...

--> Request Feminino de 50 ate 54 anos 9.009044170379639
15092 - MAGDA APARECIDA DA SILVA
13453 - IVONE TEREZINHA BASSEGIO
20206 - GISELE ADNET
26361 - LUCIA MARIA BARCELOS GOMES
12127 - DAYSE MEIRA MONTENEGRO MISERANI
20273 - REJANE MARIA WAGNITZ
26768 - NANCI VILANO DE SALVO
16861 - MARIA CRISTINA SCANDIUZZI
25390 - MONICA OPIPARI
24202 - ANDREA CRISTINA MARANGONI MUNIZ
20014 - AMILCA DA SILVA
13961 - PAULINA VIVIANA URZUA DAVIS
19117 - GEORGIA CONCEICAO MOURA BARCELOS
20399 - ANDREA ROTHFELD
15534 - GISLENE CASSIANO
26003 - ANNA LUCIA SILVA NUNES
24603 - NANCY FABIANA BENDER
12426 - LIZANDRA 0LIVEIRA PUNGIRUM
13174 - CASSIA DA PENHA PAES BENDASSOLI
23803 - ROSALI CRISTINA LEMOS
19956 - CARLA COELHO SIQUEIRA
28639 - MARIA APARECIDA DE ARAUJO FREITAS
23347 - EDILMA DE OLIVEIRA LIMA
18124 - ANA LUIZA KOELER
15984 - MARIA LAURA CASTANEDA
20762 - MONICA CRISTINA DA COSTA SANTOS
21115 - SANDRA ALVES MEDEIROS
23469 - ADRIANA SOUZA PEIXOTO
24578 - ELISABETE DOS SANTOS MACEDO
27140 - VANIA W

26268 - ANA PAULA MENDES
17859 - ROSELI BESKOW
13634 - ANA PAULA ROSA DENYS FERNANDES JULIO
15171 - MIRIAM UONO
28204 - ALEJANDRA MARIA SUAREZ
22118 - LIDUINA ELIZABETH BEZERRA
25546 - ALEXANDRA VIEIRA DA SILVA
19555 - DALVA ADRIANA DOS REIS
20523 - SORAYA RETORE KITAHARA
27297 - MARIA LUCIA DOS SANTOS MARTINS
23029 - JUSSARA CHAVES CAMARA
22969 - KATIA NARUBA DE OLIVEIRA
11991 - LUCIENE AVES DA S FERREIRA
25798 - LUSINETE LOPES
23244 - MARIA LUCIA SABELLA MUNOZ
19201 - CHRISTIANNE CALCAVECCHIA
24286 - ZENILDA RIBEIRO RODRIGUES MARQUES
27388 - SANDRA MOTA
18302 - LOURDES SUELY DOLZANIS COSTA
21696 - MONICA C N HENRIQUE
26818 - ELISABETE GOMES LINHARES
19014 - ANDREIA GOMES DE SOUZA
20397 - ADRIANA ROCHA RODRIGUES
13611 - MARIA ELISA BRASIL VIEIRA DOS SANTOS
27271 - MARISA MARIA DA COSTA
27895 - CHRISTINE SPANGLER
16882 - LENIELSA BATISTA SALES
19761 - NILSON LUIZ ASCARI
19099 - ROSA MARIA FERREIRA LIMA
16625 - ERICA RODRIGUES DOS SANTOS
16701 - ENI APARECIDA DAMAS DOS SANTOS
23175 - CL

23535 - BARBARA CARVALHO FORES DOMINGUES
12438 - HELENA RAQUEL MULLER ROCHA
16668 - TAMARA ELENA CELEDON SANHUEZA
23524 - LUDIMILA LAGO
21732 - ELISA FURTADO FERNANDES
17668 - CINTHIA FANTAZZINI VAZ BRAGA
23113 - SUELEN GONZAGA DE PAULA
20010 - NATHALIA JARDIM MACHADO HYER RANGEL
28082 - LUCILA PATRICIA NOYA
23398 - NATALIA SARRO DE ALMEIDA MELO
18678 - ERIKA FAMINI SILVA
23400 - ANA BEATRIZ MELO MANCEBO
23541 - SAMARA GONCALVES FONSECA
23203 - GABRIELLA GUIMARAES CARDOSO
21952 - MAYARA PAVESI SIEPIERSKI
25730 - JULIANA DE OLIVEIRA ZOFFOLI
16514 - AGATA FRANCISCA OLIVEIRA VIEIRA
26537 - CLAUDIA DANIELA DA SILVA CARNEIRO
15862 - IVANNA CARLA FERNANDES GUIMARAES
19237 - LUCIANA CABRAL DE OLIVEIRA SILVA
23586 - FLORENCIA AMEN
17097 - SORAYA COSTA DE OLIVEIRA MARINHO
24003 - RUTH PRISCILA DE SOUSA
23352 - TATIANE MENDES HERRERA
20899 - GABRIELLA FIUZA RAGE ZACHARIAS
24254 - ADRIANA KECIA DA SILVA RODRIGUES
24156 - TACIANA ARAUJO
27565 - TANIA ALMEIDA
18187 - EMILI SOUSA DO ESPIRITO SANTO D

22669 - MARY TUMMINELLI CARDOSO FONTES
27432 - LUCIANA VILARDO LANZILLOTTA
22811 - FABIANE GOMES FERNANDES PEREIRA MUNIZ DA COSTA
25825 - CECILIA ANDREIA ANDRADE DA SILVA
22722 - SIMONE CHRISTINA SANTANA
25588 - NILVA DONIZETI DA SILVA
21403 - BETA FORLIN
26596 - PATRICIA FERREIRA VIEIRA BRASIL
23389 - ADRIANA NOGUEIRA
15170 - CRISTIANA DAN OASHI
19096 - RAQUEL FULGENCIO GAZZOLI
23506 - VANIA CAPETTINI
24983 - MARCIA PINHEIRO DE SA
28144 - SILVIA NOEMI SANCHEZ
18965 - RENE LOPES FERREIRA
16838 - TATIANA RIBEIRO SOARES
17519 - SALETE MOREIRA GOMES
25145 - LUCIANA SHIRATORI
13008 - ADRIANA CRISTINA GALERA CONCEICAO
27267 - VERA LUCIA FERRARI
21306 - LEILA DE SOUZA DINIZ ROCHA
12224 - CLAUDIA CILENE RODRIGUES DA COSTA
12156 - ANDREIA DE CASTRO CRUZ
24797 - ADRIANA C SANCHEZ
19074 - ANALICE DE MORAIS SALDANHA DUARTE
11819 - ANDREA STANCUS STINGEL
16431 - ELYSANDRA FIGUEREDO CYPRIANO
24032 - ALEXANDRA ACEDO
16852 - CLAUDIA REGINA LIMA JACOB
20542 - MIRNA APARECIDA SOUZA CRUZ
25621 - ALEXAND

--> Request Feminino de 60 ate 64 anos 9.007315874099731
15251 - MARIA DAS GRACA DA SILVA CAIRES
16961 - SOLANGE MAGALHAES
28360 - MARTA LEONOR CERUBINI
16823 - DAISY MAGALHAES SOARES
24760 - LUIZA DOS SANTOS DA SILVA
25720 - LADY HELENA SAMPAIO UCHOA
20090 - EDNA MARCIA DE OLIVEIRA SILVA
25281 - ROSEMARY  F REIS
27652 - ILDA RODRIGUES FERREIRA
18065 - MARIA IDALIA DE ALCANTARA TANAJURA
26085 - MARCIA DA COSTA COELHO
25282 - SARA MARIA FERREIRA NEVES
16232 - SONIA REGINA DE MELO FERREIRA MENDES
18424 - MARLENE DOS SANTOS DA ROCHA
27648 - MARIA ROSALIA JOAZEIRO DE ABREU
24742 - MARIA APARECIDA RIBEIRO
21766 - SILVIA REGINA MARTINI
15601 - ELISABETH SCHETTERT
13054 - VERA LUCIA DA SILVA
27163 - YARA DE ALENCAR
18948 - NEUSA MARIA PERON
19069 - MARIA AMELIA NUNES CAMPOS
13471 - MARIA RITA DE CAIRES CORIGLIANO
16097 - SULMA MAGDALENA SANDOVAL
13994 - ADRIANA MARIA RODRIGUEZ FARALDO
12683 - REGINA COELI DE CARVALHO
26100 - ANA MARIA TAVARES PEREIRA MOTTA
23846 - ELISABETH GONCALVES DE FREIT

In [11]:
with open("temp-urls-individuais.txt", "w") as f:
    for i in url_individuais:
        f.write(i + "\n")

### Funções Utilitárias

In [5]:
# Recupera a listagem de categorias da prova do evento
def get_categorias(evento):
    
    br.open(url_base + MEIA_MARATONA)
    html = br.response().read()
    
    cat_soup = BeautifulSoup(html, "html.parser")
    select_categorias = cat_soup.find(class_ = "form-control campocategoria")
    
    if select_categorias is None:
        return None
    
    options_categorias = select_categorias.find_all("option")
    
    if options_categorias is None:
        return None
    
    options_categorias = options_categorias[1:]
    
    categorias = []
    
    for o in options_categorias:
        categorias.append(o['value'])
    
    return categorias

In [6]:
#Recupera a URL para o resultado individual de cada atleta do evento por categoria
def get_url_resultados_individuais(lista_categorias, evento):
    
    url_resultados_individuais = []

    for categoria in lista_categorias:

        #Controle de quantidade de requisição
        inicio = time()
        sleep(randint(1,10))
        espera = time() - inicio

        print('--> Request ' + categoria + ' ' + str(espera))
        
        params = {'npeito': '', 'campopesquisa': '', 'categoria': categoria}

        data = urllib.parse.urlencode(params)

        br.open(url_base + evento, data = data)
        resp = br.response().read()
        soup = BeautifulSoup(resp, "html.parser")

        div_resultados = soup.find_all(class_ = "campos")

        if div_resultados is None:
            continue

        for resultado in div_resultados:
            a = resultado.find("a")
            
            if a is None:
                continue;
                
            url_individual = a['href']
            
            url_resultados_individuais.append(url_individual)

            print(resultado.find("b").text)
        
    return url_resultados_individuais